In [1]:
%matplotlib inline

# Transfer Learning
In this assignment, we will use the weights of a network pre-trained in a particular problem as starting point to train our CNN to a different problem. As training a network from scratch is time-consuming and demands a lot of data, this is a frequent strategy, specially if both datasets (the one used for pre-training and the target) shares similar structures/elements/concepts. 

This is specially true when working with images. Most filters learned in initial convolutional layers will detect low-level elements, such as borders, corners and color blobs, which are common to most problems in the image domain. 

In this notebook, we will load the SqueezeNet architecture trained in the ImageNet dataset and fine-tune it to CIFAR-10.

## Imports

In [15]:
import os
import numpy as np
from random import sample, seed
seed(42)
np.random.seed(42)

import matplotlib.pyplot as plt
# plt.rcParams['figure.figsize'] = (15,15) # Make the figures a bit bigger

# Keras imports
from keras.layers import Input, Convolution2D, MaxPooling2D, Activation, concatenate, Dropout, GlobalAveragePooling2D, Flatten, Dense
from keras.models import Model
from keras import regularizers
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import load_img, img_to_array
from keras.datasets import cifar10
from keras.callbacks import TensorBoard
from sklearn.cross_validation import StratifiedShuffleSplit
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

#Utility to plot
def plotImages(imgList):
    for i in range(len(imgList)):
        plotImage(imgList[i])
        
        
def plotImage(img):
    fig = plt.figure(figsize=(3,3))
    ax = fig.add_subplot(111)

    ax.imshow(np.uint8(img), interpolation='nearest')
    plt.show()

## SqueezeNet definition
These methods define our architecture and load the weights obtained using ImageNet data.

In [6]:
# Fire Module Definition
sq1x1 = "squeeze1x1"
exp1x1 = "expand1x1"
exp3x3 = "expand3x3"
relu = "relu_"

def fire_module(x, fire_id, squeeze=16, expand=64):
    s_id = 'fire' + str(fire_id) + '/'

    channel_axis = 3
    
    x = Convolution2D(squeeze, (1, 1), padding='valid', name=s_id + sq1x1)(x)
    x = Activation('relu', name=s_id + relu + sq1x1)(x)

    left = Convolution2D(expand, (1, 1), padding='valid', name=s_id + exp1x1)(x)
    left = Activation('relu', name=s_id + relu + exp1x1)(left)

    right = Convolution2D(expand, (3, 3), padding='same', name=s_id + exp3x3)(x)
    right = Activation('relu', name=s_id + relu + exp3x3)(right)

    x = concatenate([left, right], axis=channel_axis, name=s_id + 'concat')
    return x

#SqueezeNet model definition
def SqueezeNet(input_shape):
    img_input = Input(shape=input_shape) #placeholder
    
    x = Convolution2D(64, (3, 3), strides=(2, 2), padding='valid', name='conv1')(img_input)
    x = Activation('relu', name='relu_conv1')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool1')(x)

    x = fire_module(x, fire_id=2, squeeze=16, expand=64)
    x = fire_module(x, fire_id=3, squeeze=16, expand=64)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool3')(x)

    x = fire_module(x, fire_id=4, squeeze=32, expand=128)
    x = fire_module(x, fire_id=5, squeeze=32, expand=128)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='pool5')(x)

    x = fire_module(x, fire_id=6, squeeze=48, expand=192)
    x = fire_module(x, fire_id=7, squeeze=48, expand=192)
    x = fire_module(x, fire_id=8, squeeze=64, expand=256)
    x = fire_module(x, fire_id=9, squeeze=64, expand=256)
    
    x = Dropout(0.5, name='drop9')(x)

    x = Convolution2D(1000, (1, 1), padding='valid', name='conv10')(x)
    x = Activation('relu', name='relu_conv10')(x)
    x = GlobalAveragePooling2D()(x)
    x = Activation('softmax', name='loss')(x)

    model = Model(img_input, x, name='squeezenet')

    # Download and load ImageNet weights
    model.load_weights('./squeezenet_weights_tf_dim_ordering_tf_kernels.h5')
    
    return model    

## CIFAR-10

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. The class are **airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck**.

In [36]:
#Load data
(trainVal_data, trainVal_label), (X_test, y_test) = cifar10.load_data()
print("Train/Val data. X: ", trainVal_data.shape, ", Y: ", trainVal_data.shape)
print("Test data. X: ", X_test.shape, ", Y: ", y_test.shape)

# Prepare the data
# ...
train_x, val_x, train_labels, val_labels = train_test_split(trainVal_data, trainVal_label, test_size=0.2, random_state=0)


Train/Val data. X:  (50000, 32, 32, 3) , Y:  (50000, 32, 32, 3)
Test data. X:  (10000, 32, 32, 3) , Y:  (10000, 1)


-----------------
## SqueezeNet with frozen layers
Our initial attempt will be to remove SqueezeNet's top layers --- responsible for the classification into ImageNet classes --- and train a new set of layers to our CIFAR-10 classes. We will also freeze the layers before `drop9`. Our architecture will be like this:

<img src="frozenSqueezeNet.png" width=70% height=70%>

In [109]:
from keras import models

squeezeNetModel = SqueezeNet((32,32,3))

#freeze layers
for layer in squeezeNetModel.layers:
    layer.trainable = False

#Add new classification layers
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

Now, we compile our model and train it:

In [112]:
model.layers[-4].trainable

True

In [113]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

In [114]:
from keras import optimizers

BATCHSIZE = 32

#Compile model
# ...
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Tensorboard callback
#tbCallBack = TensorBoard(log_dir="./logs/rafa", write_graph=True)
tbCallBack = TensorBoard(log_dir="/TransferLearning/logs/{}".format(time()), write_graph=True)

#Train model
# ...

history = model.fit_generator(datagen.flow(train_x, train_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=30, use_multiprocessing=True,
                    initial_epoch=0)

Epoch 1/30
1250/1250 [==============================] - 40s 32ms/step - loss: 2.3025 - acc: 0.1325 - val_loss: 2.2575 - val_acc: 0.1753
Epoch 2/30
1250/1250 [==============================] - 19s 16ms/step - loss: 2.2553 - acc: 0.1708 - val_loss: 2.2066 - val_acc: 0.2184
Epoch 3/30
1250/1250 [==============================] - 19s 16ms/step - loss: 2.2156 - acc: 0.1997 - val_loss: 2.1532 - val_acc: 0.2601
Epoch 4/30
1250/1250 [==============================] - 20s 16ms/step - loss: 2.1715 - acc: 0.2223 - val_loss: 2.0884 - val_acc: 0.2808
Epoch 5/30
1250/1250 [==============================] - 18s 14ms/step - loss: 2.1157 - acc: 0.2462 - val_loss: 2.0089 - val_acc: 0.3128
Epoch 6/30
1250/1250 [==============================] - 18s 14ms/step - loss: 2.0609 - acc: 0.2623 - val_loss: 1.9336 - val_acc: 0.3434
Epoch 7/30
1250/1250 [==============================] - 17s 14ms/step - loss: 2.0083 - acc: 0.2812 - val_loss: 1.8860 - val_acc: 0.3623
Epoch 8/30
1250/1250 [==========================

Finally, let's evaluate on our test set:

In [115]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(val_x, val_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

['loss', 'acc']
[1.7015583415985107, 0.4135]


-----------------
-----------------

# Training last 2 Fire Modules + classification layers
As we could see, the frozen network performed very poorly. By freezing most layers, we do not allow SqueezeNet to adapt its weights to features present in CIFAR-10.

Let's try to unfreeze the last two fire modules and train once more. The architecture will be:
<img src="partFrozenSqueezeNet.png" width=70% height=70%>

In [116]:
squeezeNetModel = SqueezeNet((32,32,3))

#freeze the mentioned layers
# ...
for layer in squeezeNetModel.layers[:-19]:
    layer.trainable = False

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(1000, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model = Model(squeezeNetModel.inputs, x, name='squeezenet_new')

In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 15, 15, 64)   1792        input_2[0][0]                    
__________________________________________________________________________________________________
relu_conv1 (Activation)         (None, 15, 15, 64)   0           conv1[0][0]                      
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 7, 7, 64)     0           relu_conv1[0][0]                 
__________________________________________________________________________________________________
fire2/sque

In [129]:
model.layers[-20].trainable

False

Now, we compile our model and train it:

In [130]:
#Compile model and train it
# ...
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

BATCHSIZE = 32

#Compile model
# ...
model.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Train model
# ...

history = model.fit_generator(datagen.flow(train_x, train_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=30, use_multiprocessing=True,
                    initial_epoch=0)

Epoch 1/30
1250/1250 [==============================] - 57s 45ms/step - loss: 3.1215 - acc: 0.1767 - val_loss: 1.8806 - val_acc: 0.3539
Epoch 2/30
1250/1250 [==============================] - 33s 26ms/step - loss: 1.8929 - acc: 0.3125 - val_loss: 1.6998 - val_acc: 0.4006
Epoch 3/30
1250/1250 [==============================] - 33s 26ms/step - loss: 1.7503 - acc: 0.3604 - val_loss: 1.6204 - val_acc: 0.4174
Epoch 4/30
1250/1250 [==============================] - 33s 26ms/step - loss: 1.6892 - acc: 0.3865 - val_loss: 1.5823 - val_acc: 0.4393
Epoch 5/30
1250/1250 [==============================] - 32s 25ms/step - loss: 1.6530 - acc: 0.4006 - val_loss: 1.5497 - val_acc: 0.4492
Epoch 6/30
1250/1250 [==============================] - 30s 24ms/step - loss: 1.6197 - acc: 0.4143 - val_loss: 1.5310 - val_acc: 0.4479
Epoch 7/30
1250/1250 [==============================] - 32s 26ms/step - loss: 1.5969 - acc: 0.4225 - val_loss: 1.5179 - val_acc: 0.4514
Epoch 8/30
1250/1250 [==========================

Finally, let's evaluate on our test set:

In [131]:
# Evaluate on validation:
# ...
print(model.metrics_names)
print(model.evaluate_generator(datagen.flow(val_x, val_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

['loss', 'acc']
[1.3927965339660644, 0.501]


-----------
-----------
-----------
# Tensorboard

Tensorboard is a visualization tool for Tensorflow. Among other things, it allows us to monitor the progress of our training, plot metrics per epochs, visualize the architecture's schematics. 

Just like for Early Stopping, we will use the [Tensorboard callback](https://keras.io/callbacks/#tensorboard) to log the information about our training. An example of usage, would be:

As your training progresses, Keras will log the metrics (e.g., loss, accuracy) to `<<LOG_DIR>>` (**make sure `<<LOG_DIR>>` is a valid directory)**. On your terminal, you will need to run Tensorboard, assign a port and access it via browser (just like jupyter).

#### ----> MAKE SURE YOU USE A DIFFERENT PORT FOR JUPYTER AND TENSORBOARD <----

### Docker
For those using docker, open a new terminal and create a new container (using the same image) running Tensorboard:

For example:

After starting Tensorboard, access it via browser on `http://localhost:<<port_container>>`.

### Anaconda
$ tensorboard --logdir=<<LOG_DIR>> --port=<<port>>

After starting Tensorboard, access it via browser on `http://localhost:<<port>>`.

-----------
-----------
-----------

# Fine-tuning all layers

What if we fine-tune all layers of SqueezeNet?
<img src="unfrozenSqueezeNet.png" width=70% height=70%>

In [144]:
squeezeNetModel = SqueezeNet((32,32,3))

for layer in squeezeNetModel.layers:
    layer.trainable = True       #by default they are all trainable, but just for clarification

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model_adam = Model(squeezeNetModel.inputs, x, name='squeezenet_new_adam')

Now, we compile our model and train it:

In [145]:
from time import time
import keras.callbacks as callbacks
BATCHSIZE = 400

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

#Compile model
# ...
model_adam.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.Adam(), metrics=["accuracy"])

#Tensorboard callback
#tbCallBack = TensorBoard(log_dir="./logs/rafa", write_graph=True)
# tbCallBack = TensorBoard(log_dir="./monitor".format(time()), write_graph=True)
tbCallBack = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

#Train model
# ...

history = model_adam.fit_generator(datagen.flow(train_x, train_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=[tbCallBack])

Epoch 1/100
100/100 [==============================] - 39s 395ms/step - loss: 1.6779 - acc: 0.4136 - val_loss: 1.1770 - val_acc: 0.6020
Epoch 2/100
100/100 [==============================] - 13s 135ms/step - loss: 1.0857 - acc: 0.6359 - val_loss: 0.9879 - val_acc: 0.6674
Epoch 3/100
100/100 [==============================] - 13s 132ms/step - loss: 0.9130 - acc: 0.6965 - val_loss: 0.8335 - val_acc: 0.7153
Epoch 4/100
100/100 [==============================] - 13s 128ms/step - loss: 0.8314 - acc: 0.7226 - val_loss: 0.8026 - val_acc: 0.7333
Epoch 5/100
100/100 [==============================] - 13s 131ms/step - loss: 0.7696 - acc: 0.7423 - val_loss: 0.7862 - val_acc: 0.7410
Epoch 6/100
100/100 [==============================] - 13s 135ms/step - loss: 0.7293 - acc: 0.7555 - val_loss: 0.7585 - val_acc: 0.7445
Epoch 7/100
100/100 [==============================] - 14s 137ms/step - loss: 0.6848 - acc: 0.7729 - val_loss: 0.7184 - val_acc: 0.7598
Epoch 8/100
100/100 [===========================

In [149]:
squeezeNetModel = SqueezeNet((32,32,3))

for layer in squeezeNetModel.layers:
    layer.trainable = True       #by default they are all trainable, but just for clarification

#Add new classification layers
# ...
x = squeezeNetModel.layers[-5].output
x = Convolution2D(10, (1, 1), padding='valid', name='new_conv10')(x)
x = Activation('relu', name='new_relu_conv10')(x)
x = GlobalAveragePooling2D()(x)
x = Activation('softmax', name='new_loss')(x)

#new Model
model_sgd = Model(squeezeNetModel.inputs, x, name='squeezenet_new_sgd')

In [ ]:
from time import time

BATCHSIZE=32

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    horizontal_flip=True,
    vertical_flip=False)

datagen.fit(train_x)

#Compile model
# ...
model_sgd.compile(loss = "sparse_categorical_crossentropy", optimizer = optimizers.SGD(lr = 0.0001, momentum=0.9, nesterov=True), metrics=["accuracy"])

#Tensorboard callback
#tbCallBack = TensorBoard(log_dir="./logs/rafa", write_graph=True)
tbCallBack = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

#Train model
# ...
history = model_sgd.fit_generator(datagen.flow(train_x, train_labels, batch_size=BATCHSIZE),
                    validation_data=datagen.flow(val_x, val_labels, batch_size=BATCHSIZE),
                    validation_steps=len(val_x)/BATCHSIZE,
                    steps_per_epoch=len(train_x)/BATCHSIZE, 
                    epochs=100, use_multiprocessing=True,
                    initial_epoch=0,
                    callbacks=[tbCallBack])

Epoch 1/100
1250/1250 [==============================] - 64s 51ms/step - loss: 2.0079 - acc: 0.2782 - val_loss: 1.5645 - val_acc: 0.4571
Epoch 2/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.5412 - acc: 0.4484 - val_loss: 1.2906 - val_acc: 0.5447
Epoch 3/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.3551 - acc: 0.5256 - val_loss: 1.1827 - val_acc: 0.5893
Epoch 4/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.2488 - acc: 0.5639 - val_loss: 1.1070 - val_acc: 0.6075
Epoch 5/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.1749 - acc: 0.5924 - val_loss: 1.0586 - val_acc: 0.6341
Epoch 6/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.1223 - acc: 0.6146 - val_loss: 1.0202 - val_acc: 0.6497
Epoch 7/100
1250/1250 [==============================] - 40s 32ms/step - loss: 1.0756 - acc: 0.6317 - val_loss: 0.9779 - val_acc: 0.6569
Epoch 8/100
1250/1250 [==================

Finally, let's evaluate on our validation set:

In [ ]:
# Evaluate on validation
# ...

print(model_sgd.metrics_names)
print(model_sgd.evaluate_generator(datagen.flow(val_x, val_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

print(model_adam.metrics_names)
print(model_adam.evaluate_generator(datagen.flow(val_x, val_labels, batch_size=BATCHSIZE), steps=len(val_x)/BATCHSIZE))

In [ ]:
# Evaluate your best model on test
# ...

## Saving the model
Now that we are working on more complex tasks and our trainings are starting to take more time it is usually a good idea to save the trained model from time to time. [Keras has a lot of ways of saving and loading the model](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model), but in this exercise we will use the simplest of them all: `model.save()`. It saves the architecture, the weights, the choice of loss function/optimizer/metrics and even the current state of the training, so you can resume your training later.

In [ ]:
model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

## Loading a model
Once we have our model trained, we can load it using:

In [ ]:
from keras.models import load_model

del model  # Will delete model, only to check if load_model is working

# returns a compiled model identical to the previous one
model = load_model('my_model.h5')

# evaluate test set again... should give us the same result
# ...
print('Test loss:', score[0])
print('Test accuracy (NORMALIZED):', score[1])